In [ ]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import ray
import numpy as np
warnings.filterwarnings('ignore')
config = ppo.DEFAULT_CONFIG.copy()
config

In [ ]:
MAX_EPS = 100
agent_name = 'Blue'

def wrap(env):
    return RLlibWrapper(agent_name="Blue", env=env)


def evaluate(steps):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    obs = []
    #print(f'using CybORG v{cyborg_version}, {scenario}\n')
    for num_steps in steps:
        for red_agent in [B_lineAgent, RedMeanderAgent]:

            cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
            wrapped_cyborg = wrap(cyborg)

            observation = wrapped_cyborg.reset()
            obs.append(observation)
            # observation = cyborg.reset().observation

            action_space = wrapped_cyborg.get_action_space(agent_name)
            # action_space = cyborg.get_action_space(agent_name)
            total_reward = []
            actions = []
            for i in range(MAX_EPS):
                r = []
                a = []
                # cyborg.env.env.tracker.render()
                for j in range(num_steps):
                    action = trainer.compute_single_action(observation)
                    action_vec = np.zeros(145)
                    action_vec[int(action)] = 1
                    #action = agent.get_action(observation, action_space)
                    observation, rew, done, info = wrapped_cyborg.step(action)
                    obs.append(observation)
                    actions.append(action_vec)
                    # result = cyborg.step(agent_name, action)
                    r.append(rew)
                    # r.append(result.reward)
                    a.append((str(cyborg.get_last_action('Blue')), str(cyborg.get_last_action('Red'))))
                total_reward.append(sum(r))
                # observation = cyborg.reset().observation
                observation = wrapped_cyborg.reset()
            print(f'Average reward for red agent {red_agent.__name__} and steps {num_steps} is: {mean(total_reward):.1f} with a standard deviation of {stdev(total_reward):.1f}')
            return mean(total_reward), np.mean(np.array(obs), axis=0),  np.mean(np.array(actions), axis=0)

In [3]:
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
from ray.rllib.models import ModelCatalog
from ray.tune.registry import register_env
from ray.rllib.agents import ppo
import inspect
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper

def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

import subprocess
import json
import os

batch_size = 4000
# Set up CybORG
register_env(name="CybORG", env_creator=env_creator)
config = ppo.DEFAULT_CONFIG.copy()


from shutil import make_archive
allrewards = []

config.update({"num_gpus": 1,"num_workers": 20,
                'num_cpus_per_worker':0.5,
                "framework": "tf2",
                "train_batch_size": batch_size,
                "horizon": 100,
                "gamma": 0.95,
                "model": {
                    "fcnet_hiddens": [512, 512],
                    "fcnet_activation": "relu",
                },

                }) 
trainer = ppo.PPOTrainer(config=config, env="CybORG")

reward = []
m = -30
for i in range(int(750)):
    results_dict=trainer.train()
    print_results(results_dict)
    if m < results_dict["episode_reward_mean"]:
        reward.append(results_dict["episode_reward_mean"])
        print(trainer.save("bline_ppo"))
        m = results_dict["episode_reward_mean"]
        


(RolloutWorker pid=23942) 2022-08-10 09:29:07,817	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=23957) 2022-08-10 09:29:07,962	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=23964) 2022-08-10 09:29:08,041	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=23940) 2022-08-10 09:29:07,986

   1 	r_mean: -681.1 	r_max: -101.8 	r_min: -1144.8
   2 	r_mean: -527.8 	r_max: -22.3 	r_min: -1144.8
   3 	r_mean: -420.7 	r_max: -18.1 	r_min: -1143.7
   4 	r_mean: -284.0 	r_max: -14.8 	r_min: -1069.4
   5 	r_mean: -278.8 	r_max: -14.8 	r_min: -890.8
   6 	r_mean: -263.7 	r_max: -14.8 	r_min: -890.8
   7 	r_mean: -244.7 	r_max: -69.8 	r_min: -890.8
   8 	r_mean: -219.2 	r_max: -69.8 	r_min: -675.8
   9 	r_mean: -215.9 	r_max: -69.8 	r_min: -522.8
  10 	r_mean: -216.0 	r_max: -18.6 	r_min: -522.8
  11 	r_mean: -204.9 	r_max: -18.6 	r_min: -522.8
  12 	r_mean: -193.2 	r_max: -18.6 	r_min: -473.5
  13 	r_mean: -186.4 	r_max: -45.8 	r_min: -473.5
  14 	r_mean: -185.1 	r_max: -93.4 	r_min: -473.5
  15 	r_mean: -169.7 	r_max: -33.8 	r_min: -467.8
  16 	r_mean: -160.3 	r_max: -12.8 	r_min: -347.2
  17 	r_mean: -152.3 	r_max: -12.8 	r_min: -347.2
  18 	r_mean: -153.4 	r_max: -15.8 	r_min: -268.8
  19 	r_mean: -148.3 	r_max: -15.8 	r_min: -268.8
  20 	r_mean: -140.7 	r_max: -23.8 	r_min: -2

 168 	r_mean: -36.7 	r_max: -8.6 	r_min: -215.8
 169 	r_mean: -42.7 	r_max: -8.6 	r_min: -193.7
 170 	r_mean: -45.4 	r_max: -9.1 	r_min: -174.6
 171 	r_mean: -45.2 	r_max: -8.2 	r_min: -161.5
 172 	r_mean: -44.0 	r_max: -8.2 	r_min: -170.1
 173 	r_mean: -43.0 	r_max: -9.1 	r_min: -172.4
 174 	r_mean: -41.5 	r_max: -9.6 	r_min: -178.1
 175 	r_mean: -40.8 	r_max: -9.6 	r_min: -194.2
 176 	r_mean: -41.7 	r_max: -9.6 	r_min: -194.2
 177 	r_mean: -41.8 	r_max: -9.1 	r_min: -178.0
 178 	r_mean: -39.9 	r_max: -6.9 	r_min: -242.8
 179 	r_mean: -42.3 	r_max: -6.9 	r_min: -242.8
 180 	r_mean: -42.8 	r_max: -8.4 	r_min: -243.7
 181 	r_mean: -38.7 	r_max: -8.4 	r_min: -243.7
 182 	r_mean: -36.1 	r_max: -9.3 	r_min: -165.7
 183 	r_mean: -38.6 	r_max: -9.6 	r_min: -165.7
 184 	r_mean: -39.9 	r_max: -9.6 	r_min: -167.6
 185 	r_mean: -37.4 	r_max: -9.6 	r_min: -167.6
 186 	r_mean: -38.0 	r_max: -9.8 	r_min: -167.6
 187 	r_mean: -44.4 	r_max: -9.4 	r_min: -148.8
 188 	r_mean: -45.3 	r_max: -9.1 	r_min:

 342 	r_mean: -33.2 	r_max: -9.7 	r_min: -164.8
 343 	r_mean: -36.9 	r_max: -9.2 	r_min: -164.8
 344 	r_mean: -36.8 	r_max: -9.2 	r_min: -241.7
 345 	r_mean: -40.6 	r_max: -9.2 	r_min: -241.7
 346 	r_mean: -34.8 	r_max: -9.5 	r_min: -182.7
 347 	r_mean: -47.0 	r_max: -9.5 	r_min: -182.7
 348 	r_mean: -44.1 	r_max: -8.5 	r_min: -167.7
 349 	r_mean: -43.9 	r_max: -8.4 	r_min: -167.9
 350 	r_mean: -43.3 	r_max: -8.4 	r_min: -167.9
 351 	r_mean: -34.3 	r_max: -8.4 	r_min: -139.8
 352 	r_mean: -34.8 	r_max: -9.0 	r_min: -165.8
 353 	r_mean: -35.4 	r_max: -9.0 	r_min: -206.8
 354 	r_mean: -36.5 	r_max: -9.7 	r_min: -206.8
 355 	r_mean: -36.5 	r_max: -9.9 	r_min: -238.7
 356 	r_mean: -35.4 	r_max: -9.8 	r_min: -238.7
 357 	r_mean: -38.7 	r_max: -8.8 	r_min: -206.8
 358 	r_mean: -39.2 	r_max: -8.8 	r_min: -206.8
 359 	r_mean: -43.8 	r_max: -8.8 	r_min: -206.8
 360 	r_mean: -39.2 	r_max: -9.8 	r_min: -206.2
 361 	r_mean: -39.3 	r_max: -9.8 	r_min: -168.2
 362 	r_mean: -39.2 	r_max: -9.8 	r_min:

 513 	r_mean: -36.6 	r_max: -9.8 	r_min: -153.3
 514 	r_mean: -38.7 	r_max: -9.3 	r_min: -155.8
 515 	r_mean: -39.2 	r_max: -8.5 	r_min: -223.8
 516 	r_mean: -36.0 	r_max: -8.5 	r_min: -223.8
 517 	r_mean: -35.2 	r_max: -8.5 	r_min: -223.8
 518 	r_mean: -34.5 	r_max: -9.8 	r_min: -311.7
 519 	r_mean: -33.3 	r_max: -9.4 	r_min: -311.7
 520 	r_mean: -35.9 	r_max: -9.4 	r_min: -178.7
 521 	r_mean: -37.3 	r_max: -9.4 	r_min: -178.7
 522 	r_mean: -39.0 	r_max: -10.1 	r_min: -178.7
 523 	r_mean: -34.0 	r_max: -9.8 	r_min: -162.8
 524 	r_mean: -34.2 	r_max: -9.8 	r_min: -190.8
 525 	r_mean: -34.3 	r_max: -9.8 	r_min: -237.7
 526 	r_mean: -37.0 	r_max: -9.8 	r_min: -237.7
 527 	r_mean: -39.0 	r_max: -9.9 	r_min: -237.7
 528 	r_mean: -32.5 	r_max: -9.6 	r_min: -151.8
 529 	r_mean: -28.9 	r_max: -9.6 	r_min: -137.7
 530 	r_mean: -31.4 	r_max: -10.1 	r_min: -160.6
 531 	r_mean: -37.0 	r_max: -9.6 	r_min: -160.6
 532 	r_mean: -47.3 	r_max: -9.6 	r_min: -988.7
 533 	r_mean: -48.7 	r_max: -9.7 	r_mi

 684 	r_mean: -39.7 	r_max: -9.8 	r_min: -351.6
 685 	r_mean: -41.2 	r_max: -10.0 	r_min: -351.6
 686 	r_mean: -40.4 	r_max: -9.8 	r_min: -184.8
 687 	r_mean: -36.6 	r_max: -8.8 	r_min: -164.0
 688 	r_mean: -41.7 	r_max: -8.8 	r_min: -172.4
 689 	r_mean: -40.1 	r_max: -8.7 	r_min: -209.1
 690 	r_mean: -35.3 	r_max: -8.7 	r_min: -209.1
 691 	r_mean: -30.5 	r_max: -8.7 	r_min: -152.7
 692 	r_mean: -32.5 	r_max: -9.8 	r_min: -152.7
 693 	r_mean: -36.0 	r_max: -9.8 	r_min: -224.8
 694 	r_mean: -32.0 	r_max: -9.8 	r_min: -224.8
 695 	r_mean: -29.4 	r_max: -9.7 	r_min: -86.3
 696 	r_mean: -34.6 	r_max: -9.7 	r_min: -152.8
 697 	r_mean: -40.0 	r_max: -9.8 	r_min: -198.8
 698 	r_mean: -41.2 	r_max: -9.8 	r_min: -198.8
 699 	r_mean: -39.5 	r_max: -9.8 	r_min: -184.8
 700 	r_mean: -40.5 	r_max: -9.8 	r_min: -150.8
 701 	r_mean: -42.1 	r_max: -9.8 	r_min: -170.3
 702 	r_mean: -42.8 	r_max: -10.5 	r_min: -170.3
 703 	r_mean: -43.8 	r_max: -10.5 	r_min: -170.3
 704 	r_mean: -39.3 	r_max: -9.8 	r_mi

In [4]:
np.save('b_line_reward.npy', np.array(reward))

In [ ]:
ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update({"num_gpus": 0,"num_workers": 0,
    "framework": "tf2",
    "model": {
        "fcnet_hiddens": [512, 512],
        "fcnet_activation": "relu",
    }})

b_line = PPOTrainer(config=ppo_config,env="CybORG")
b_line.restore("bline_ppo/checkpoint_000132/checkpoint-132")

In [ ]:
132 	r_mean: -25.9 	r_max: -8.5 	r_min: -151.1
bline_ppo/checkpoint_000132/checkpoint-132

In [ ]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

import subprocess
import json
import os

batch_size = 4000
# Set up CybORG
register_env(name="CybORG", env_creator=env_creator)
config = ppo.DEFAULT_CONFIG.copy()

from shutil import make_archive
allrewards = []
for g in [0.9]:
    config.update({"num_gpus": 1,"num_workers": 20,
                    # Also, use "framework: tf2" for tfe eager execution.
                    "framework": "tf2",
                    "train_batch_size": batch_size,
                    "horizon": 100,
                    "gamma": g,
                    "model": {
                        "fcnet_hiddens": [512, 512],
                        "fcnet_activation": "relu",
                    },
                   
                    }) 
    trainer = ppo.PPOTrainer(config=config, env="CybORG")

    reward = []
    novel_obs = []
    novel_actions = []
    for i in range(200):
        results_dict = trainer.train()
        r, o, a = evaluate([100])
        reward.append(r)
        novel_obs.append(o)
        novel_actions.append(a)
    print(g)
    removed = 0
    result = subprocess.run(['ls', "/ppo"], stdout=subprocess.PIPE)
    for j, name in enumerate(str(result.stdout)[2:].split('\\n')[0:-1]):
        f = open(os.path.join("/ppo", name))
        try:
            json.load(f)
        except ValueError as err:
            os.remove(os.path.join("/ppo", name)) 
            removed += 1
    print('Removed ' + str(removed) + ' files, of ' + str(j) + 'files')
    make_archive("ppo", 'zip', "/ppo")
    allrewards.append(reward)
    np.save('ppo_reward.npy', np.array(reward))
    np.save('ppo_novel_obs.npy', np.stack(novel_obs))
    np.save('ppo_novel_actions.npy', np.stack(novel_actions))
    #if i % int((1e6/batch_size)/20) == 0:
    #    evaluate([100])
#results = trainer.train()
#print_results(results)
novel

In [ ]:
np.save('ppo_reward.npy', np.array(reward))
np.save('ppo_novel_obs.npy', np.stack(novel_obs))
np.save('ppo_novel_actions.npy', np.stack(novel_actions))

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.plot(np.load('85_data_a_evals.npy'), label="Elite - (100 Iter Evaluation)")
plt.plot(np.load('85_data_a_maxs.npy'), label="Elite - (25 Iter Evaluation)")
plt.title('Evaluation Return With Different Evaluation Iteration Counts')
plt.xlabel('Iteration')
plt.ylabel('Reward')
plt.legend(loc='lower right')
plt.xlim([50,200])
plt.ylim([-60,-15])
plt.show()

In [ ]:
print(np.mean(np.load('85_data_a_evals.npy')))
print(np.mean(np.load('85_data_a_maxs.npy')))


In [ ]:
std = []
for t in r:
    if t == []:
        break
    std.append(stdev(t))
    #print(stdev(t))
    
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
plt.plot(range(2,23,2), std)
plt.title('Standard Deviation Over Trails ')
plt.xlabel('Number Evaluations')
plt.ylabel('STD between Trails')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.boxplot(r[1:], labels=[4,6,8,10,12,14,16,18,20,22,24])
plt.title('Box Plot For Evaluation Count Experiment')
plt.xlabel('Number Evaluations')
plt.ylabel('Reward')
plt.xlim([0,11])
plt.ylim([-60,-15])
plt.show()

In [ ]:
from math import factorial
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(111)
gammas = [0.99, 0.9, 0.8, 0.7]
for i, l in enumerate(allrewards):
    ax1.plot(allrewards[i], label='gamma = ' + str(gammas[i]))

plt.legend(loc='upper left')
plt.title('Episode Length (passed though a Savitzky Golay filter)')
plt.xlabel('Episode')
plt.ylabel('Episode Length (timesteps)')
plt.show()

In [ ]:
from math import factorial
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(111)
gammas = [.8]
for i, l in enumerate(allrewards):
    ax1.plot(allrewards[i], label='gamma = ' + str(gammas[i]))

plt.legend(loc='upper left')
plt.title('Mean Reward In Training Of PPO')
plt.xlabel('Batch')
plt.ylabel('Mean Reward')
plt.show()

In [ ]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents import ppo
from ray.rllib.agents.trainer import Trainer
trainer = ppo.PPOTrainer(config={"env": "CartPole-v0", "num_workers": 0, 'log_level': 'DEBUG'})
for i in range(2):
    trainer.train()

In [ ]:
from ray.rllib.agents import ppo
ppo.DEFAULT_CONFIG.copy()

In [ ]:
import inspect

def wrap(env):
    return RLlibWrapper(agent_name="Blue", env=env)


path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'


cyborg = CybORG(path, 'sim', agents={'Red': B_lineAgent})
wrapped_cyborg = wrap(cyborg)

observation = wrapped_cyborg.reset()
#observation = cyborg.reset()

for i in range(100):
    observation, rew, done, info = wrapped_cyborg.step(0)

print(observation.shape)



In [ ]:
import matplotlib.pyplot as plt
import umap
from matplotlib import animation, rc
import numpy as np
import imageio
import os

path = 'gagif'
if not os.path.exists('gagif'):
    os.mkdir(path)
evals_ppo = np.load('ppo_reward.npy')
evals_ga = np.load('8_data_s_evals.npy') 
whole = []
whole.append(np.load('8_data_s_archive.npy'))
whole.append(np.load('ppo_novel_obs.npy'))
#Comment out umap_embedding if adjusting plots after first run, its non-determeisitic 
reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1)
umap_embedding = reducer.fit_transform(np.concatenate(whole))
animation_frames = 200
num_ga_samples = np.load('8_data_s_archive.npy').shape[0]
start = 0
num_timesteps = 200
filenames = []
for i in range(num_timesteps): 
    fig = plt.figure(figsize=(12, 8), dpi=100)
    i_line = int(i*(num_timesteps/animation_frames))
    i_scatter = int(i*(num_ga_samples/animation_frames)) #number of uMAP points to show for GA
    ax = plt.subplot(1, 2, 1)
    ax.set_xlim([-6,14])
    ax.set_ylim([-4,20]) 
    ax.set_title('uMAP of Action Embedding')
    ax1 = plt.subplot(1, 2, 2)
    ax1.set_xlim([0,200])
    ax1.set_ylim([-80,-15])
    ax1.set_title('Reward Accumilation')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Reward')
    ax1.plot(evals_ga[:i_line+1], label="GA State Novelty - Elite Evaluation")
    ax1.plot(evals_ppo[:i_line+1], label="PPO - Evaluation")
    ax1.legend(loc='lower right')
    ax.scatter(umap_embedding[0:i_scatter+1,0], umap_embedding[0:i_scatter+1,1], alpha=0.1)
    ax.scatter(umap_embedding[num_ga_samples:num_ga_samples+i+1,0], umap_embedding[num_ga_samples:num_ga_samples+i+1,1], alpha=0.1)
    plt.savefig(path+'/'+str(i)+'.png')
    filenames.append(path+'/'+str(i)+'.png')

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('cage2statenovelty.gif', images, duration=0.1)

In [ ]:
plt.show()

In [ ]:
np.load('ppo_novel_obs.npy').shape

In [ ]:
with open('fml.txt') as f:
    lines = f.readlines()
reward = []
for l in lines:
    if "eps 100 is: " in l: 
        reward.append(float((l[59:64])))

In [ ]:
import numpy as np
np.load('85_data_a_archive.npy').shape[0]

In [ ]:
from gym import spaces
observation_space = spaces.Discrete(50)
observation_space[0]

	                                                 Validated Test Scores
                                    30 steps	           50 steps	            100 steps
	                         B-line	Meander	Sleep	B-line	Meander	Sleep	B-line	Meander	Sleep
H PPO w Cur	 -30.07 ± 0.67	 -2.63	-2.28	0	    -4.71	-3.60	0	    -11.01	-5.84	0
H PPO	     -49.35 ± 0.37	 -3.87	-3.27	0	    -6.81	-6.66	0	    -14.06	-14.68  0
DDQN w RNN	 -119.01 ± 2.29	 -6.87	-6.00	0	    -12.67	-16.87	0	    -27.47	-49.13	0
DDQN	     -213.76 ± 6.45	 -8.10	-9.39	0	    -19.22	-30.62	0	    -59.66	-86.77	0